In [ ]:
#import shutil
#shutil.rmtree('/content/sample_data', ignore_errors=True)

# Microsoft XtremeDistil RoBERTa

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import tempfile
import subprocess
import sys

# IMPORT LIBRARIES AND SETUP

In [ ]:
def install_requirements():
    required_packages = [
        'transformers[torch]',
        'datasets',
        'torch',
        'pandas',
        'scikit-learn',
        'onnx',
        'onnxruntime',
        'optimum[onnxruntime]',  # This enables ORTModelForSequenceClassification
        'tensorflow',
    ]

    for package in required_packages:
        try:
            __import__(package.split('[')[0])
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [ ]:
print("Checking and installing dependencies...")
install_requirements()

Checking and installing dependencies...
Installing scikit-learn...


In [ ]:
try:
    from optimum.onnxruntime import ORTModelForSequenceClassification
    from optimum.onnxruntime.configuration import OptimizationConfig
    ONNX_AVAILABLE = True
except ImportError:
    print("Optimum ONNX Runtime not available, ONNX export will be limited")
    ONNX_AVAILABLE = False

Optimum ONNX Runtime not available, ONNX export will be limited


In [ ]:
print("All dependencies loaded successfully!")

All dependencies loaded successfully!


# LOAD AND PREPARE DATASET

In [ ]:
def load_dataset(csv_path='dataset.csv'):
    print(f"Loading dataset from {csv_path}...")

    try:
        df = pd.read_csv(csv_path)
        print(f"Dataset loaded successfully. Shape: {df.shape}")

        # Validate required columns
        required_cols = ['text', 'label']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        # Clean and validate data
        df = df.dropna(subset=['text', 'label'])
        df['text'] = df['text'].astype(str)
        df['label'] = df['label'].astype(int)

        # Check label range - XtremeDistil can handle multi-class classification
        unique_labels = sorted(df['label'].unique())
        num_labels = len(unique_labels)

        # Ensure labels are sequential starting from 0
        if unique_labels != list(range(num_labels)):
            print("Warning: Labels are not sequential starting from 0. Remapping...")
            label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}
            df['label'] = df['label'].map(label_mapping)
            print(f"Label mapping: {label_mapping}")

        print(f"Dataset validation complete. Clean shape: {df.shape}")
        print(f"Number of classes: {num_labels}")

        # Updated label descriptions for NSFW/Safe classification
        if num_labels == 2:
            print("Binary classification detected - assuming NSFW/Safe task")
            print("Label distribution:")
            label_counts = df['label'].value_counts().sort_index()
            for label, count in label_counts.items():
                label_name = "Safe" if label == 0 else "NSFW"
                print(f"  {label} ({label_name}): {count} samples")
        else:
            print(f"Label distribution:\n{df['label'].value_counts().sort_index()}")

        return df

    except FileNotFoundError:
        print(f"Error: Dataset file '{csv_path}' not found!")
        print("Creating a sample NSFW/Safe classification dataset for demonstration...")
        return create_sample_dataset()

In [ ]:
def create_sample_dataset():
    sample_data = {
        'text': [
            # Safe content (label 0) - combining positive and neutral examples
            'I love this product, it works perfectly!',
            'This is an amazing experience, highly recommended!',
            'Great quality and fast delivery service.',
            'Excellent customer support and user-friendly interface.',
            'Outstanding performance and great value for money.',
            'The product is okay, nothing special about it.',
            'Average quality, meets basic requirements.',
            'It works as described, no complaints.',
            'Standard features, typical for this price range.',
            'Normal delivery time, packaging was fine.',
            'Me encanta este producto, funciona perfectamente.',
            'Excelente calidad y servicio rápido.',
            'El producto está bien, nada especial.',
            'Calidad promedio, cumple con lo básico.',
            'Napakaganda ng produktong ito, sulit na sulit.',
            'Magandang serbisyo at mabilis na delivery.',
            'Okay lang ang produkto, walang masama.',
            'Standard quality, normal lang.',
            'Produit excellent, je le recommande vivement.',
            'Service client formidable et livraison rapide.',
            'Produit correct, sans plus.',
            'This is a family-friendly movie everyone can enjoy.',
            'Educational content about science and technology.',
            'Safe workplace environment and policies.',
            'Appropriate content for all age groups.',

            # NSFW content (label 1) - inappropriate/explicit content
            'This contains explicit adult content not suitable for minors.',
            'Inappropriate sexual references and mature themes.',
            'Contains graphic violence and disturbing imagery.',
            'Adult-oriented material with explicit language.',
            'Mature content with sexual themes and nudity.',
            'Violent and disturbing content not for children.',
            'Explicit adult entertainment material.',
            'Contains profanity and inappropriate behavior.',
            'Mature themes including alcohol and substance abuse.',
            'Graphic content with adult situations.',
            'Inappropriate workplace behavior and harassment.',
            'Explicit material containing adult themes.',
            'Disturbing content with mature subject matter.',
            'Adult content with sexual references.',
            'Inappropriate material for general audiences.',
            'Mature content with explicit themes.',
            'Adult-oriented discussions and topics.',
            'Inappropriate behavior and language.',
            'Explicit content not suitable for work.',
            'Mature themes and adult situations.',
        ],
        'label': (
            [0] * 25 +  # 25 safe content samples (combining positive/neutral)
            [1] * 20    # 20 NSFW content samples
        )
    }

    # Safety check to ensure equal lengths
    if len(sample_data['text']) != len(sample_data['label']):
        print(f"WARNING: Length mismatch! Text: {len(sample_data['text'])}, Label: {len(sample_data['label'])}")
        min_len = min(len(sample_data['text']), len(sample_data['label']))
        sample_data['text'] = sample_data['text'][:min_len]
        sample_data['label'] = sample_data['label'][:min_len]

    df = pd.DataFrame(sample_data)
    df.to_csv('dataset.csv', index=False)
    print("Sample NSFW/Safe classification dataset created and saved as 'dataset.csv'")
    print(f"Dataset shape: {df.shape}")
    print("Label mapping: 0 = Safe, 1 = NSFW")
    return df

In [ ]:
def split_dataset(df, test_size=0.2, random_state=42):
    print(f"Splitting dataset: {test_size*100}% for validation...")

    X_train, X_val, y_train, y_val = train_test_split(
        df['text'].tolist(),
        df['label'].tolist(),
        test_size=test_size,
        random_state=random_state,
        stratify=df['label']
    )

    print(f"Training set: {len(X_train)} samples")
    print(f"Validation set: {len(X_val)} samples")

    return X_train, X_val, y_train, y_val

# TOKENIZATION AND PREPROCESSING

In [ ]:
def create_tokenized_datasets(X_train, X_val, y_train, y_val, model_name):
    """Tokenize the datasets using the XtremeDistil tokenizer"""
    print("Loading XtremeDistil tokenizer and creating tokenized datasets...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # XtremeDistil uses BERT-style tokenization
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.unk_token if tokenizer.unk_token is not None else '[PAD]'

    # Create datasets
    train_dataset = Dataset.from_dict({
        'text': X_train,
        'labels': y_train
    })

    val_dataset = Dataset.from_dict({
        'text': X_val,
        'labels': y_val
    })

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            truncation=True,
            padding=False,  # Will be handled by data collator
            max_length=128  # XtremeDistil works well with shorter sequences
        )

    # Tokenize datasets
    train_tokenized = train_dataset.map(tokenize_function, batched=True)
    val_tokenized = val_dataset.map(tokenize_function, batched=True)

    print("Tokenization complete!")
    print(f"Sample tokenized text length: {len(train_tokenized[0]['input_ids'])}")

    return train_tokenized, val_tokenized, tokenizer

# MODEL TRAINING

In [ ]:
def train_model(train_dataset, val_dataset, tokenizer, model_name, output_dir, num_labels):
    """Train the XtremeDistil model for multilingual classification"""
    print("Initializing XtremeDistil model for multilingual classification training...")

    # Load model with appropriate number of labels
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        problem_type="single_label_classification"
    )

    # Ensure model uses the correct pad_token_id
    if tokenizer.pad_token_id is not None:
        model.config.pad_token_id = tokenizer.pad_token_id

    # Data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Training arguments optimized for XtremeDistil
    training_args_dict = {
        'output_dir': output_dir,
        'num_train_epochs': 4,  # Slightly more epochs due to distilled model
        'per_device_train_batch_size': 16,  # Larger batch size (smaller model)
        'per_device_eval_batch_size': 16,
        'learning_rate': 5e-5,  # Higher learning rate for distilled models
        'weight_decay': 0.01,
        'warmup_ratio': 0.1,  # Warmup ratio instead of steps
        'logging_dir': f'{output_dir}/logs',
        'logging_steps': 10,
        'save_total_limit': 2,
        'load_best_model_at_end': True,
        'metric_for_best_model': "eval_f1_macro",
        'greater_is_better': True,
        'report_to': [],
        'seed': 42,
        'dataloader_num_workers': 0,
        'remove_unused_columns': True,
        'fp16': True,  # Mixed precision for efficiency
        'dataloader_pin_memory': False,
        'gradient_checkpointing': False,  # XtremeDistil is small enough
    }

    # Add version-specific parameters
    if hasattr(TrainingArguments, 'eval_strategy'):
        training_args_dict['eval_strategy'] = "epoch"
        training_args_dict['save_strategy'] = "epoch"
    else:
        training_args_dict['evaluation_strategy'] = "epoch"
        training_args_dict['save_strategy'] = "epoch"

    training_args = TrainingArguments(**training_args_dict)

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred

        # Handle different prediction formats
        if isinstance(predictions, tuple):
            predictions = predictions[0]

        # Convert to numpy array if it's a tensor
        if hasattr(predictions, 'numpy'):
            predictions = predictions.numpy()

        predictions = np.argmax(predictions, axis=1)

        # Calculate metrics
        accuracy = accuracy_score(labels, predictions)
        f1_macro = f1_score(labels, predictions, average='macro')
        f1_weighted = f1_score(labels, predictions, average='weighted')

        return {
            'accuracy': accuracy,
            'f1_macro': f1_macro,
            'f1_weighted': f1_weighted,
        }

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("Starting training...")
    trainer.train()

    # Save the best model
    print(f"Saving model to {output_dir}...")
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

    return trainer, model

# MODEL EVALUATION

In [ ]:
def evaluate_model(trainer, X_val, y_val, output_dir, num_labels):
    print("Evaluating model performance...")

    # Get predictions
    eval_results = trainer.evaluate()

    # Get detailed predictions for classification report
    predictions = trainer.predict(trainer.eval_dataset)

    # Extract predictions from the prediction object
    if hasattr(predictions, 'predictions'):
        preds = predictions.predictions
    else:
        preds = predictions[0]

    # Convert to numpy array if it's a tensor
    if hasattr(preds, 'numpy'):
        preds = preds.numpy()

    y_pred = np.argmax(preds, axis=1)

    # Generate classification report with updated target names
    if num_labels == 2:
        target_names = ['Safe', 'NSFW']  # This part works correctly
    elif num_labels == 3:
        target_names = ['Positive', 'Neutral', 'Negative']
    else:
        target_names = [f'Class_{i}' for i in range(num_labels)]

    report = classification_report(
        y_val,
        y_pred,
        target_names=target_names,
        digits=4
    )

    # Calculate additional metrics
    accuracy = accuracy_score(y_val, y_pred)
    f1_macro = f1_score(y_val, y_pred, average='macro')
    f1_weighted = f1_score(y_val, y_pred, average='weighted')

    # Updated metrics text for NSFW/Safe classification
    metrics_text = f"""XtremeDistil NSFW/Safe Classification Model Evaluation Results
{'='*70}

Model: microsoft/xtremedistil-l6-h384-uncased
Task: NSFW/Safe Content Classification (Binary Classification)
Architecture: 6 layers, 384 hidden units (Distilled)

Classification Labels:
- 0: Safe Content
- 1: NSFW Content

Performance Metrics:
{'-'*30}
Accuracy: {accuracy:.4f}
F1-Score (Macro): {f1_macro:.4f}
F1-Score (Weighted): {f1_weighted:.4f}

Classification Report:
{report}

Training Results:
{'-'*30}
"""

    for key, value in eval_results.items():
        if isinstance(value, (int, float)):
            metrics_text += f"{key}: {value:.4f}\n"

    # Save metrics
    os.makedirs('metrics', exist_ok=True)
    metrics_path = 'metrics/xtremedistil_nsfw_safe_metrics.txt'

    with open(metrics_path, 'w') as f:
        f.write(metrics_text)

    print(f"Metrics saved to {metrics_path}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")

    return accuracy, report

# ONNX EXPORT

In [ ]:
def export_to_onnx(model_dir, onnx_path):
    print("Exporting XtremeDistil NSFW/Safe classification model to ONNX format...")

    try:
        # Load the trained model
        model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        tokenizer = AutoTokenizer.from_pretrained(model_dir)

        # Create dummy input with NSFW/Safe classification sample
        dummy_input = tokenizer(
            "This is a sample text for NSFW/Safe classification ONNX export",
            return_tensors="pt",
            max_length=128,
            padding="max_length",
            truncation=True
        )

        # Export to ONNX with optimization for XtremeDistil
        os.makedirs(os.path.dirname(onnx_path), exist_ok=True)
        torch.onnx.export(
            model,
            tuple(dummy_input.values()),
            onnx_path,
            export_params=True,
            opset_version=14,
            do_constant_folding=True,
            input_names=['input_ids', 'attention_mask'],
            output_names=['logits'],
            dynamic_axes={
                'input_ids': {0: 'batch_size', 1: 'sequence'},
                'attention_mask': {0: 'batch_size', 1: 'sequence'},
                'logits': {0: 'batch_size'}
            }
        )

        print(f"ONNX model exported to: {onnx_path}")

        # Get model size
        model_size = os.path.getsize(onnx_path) / (1024 * 1024)
        print(f"ONNX model size: {model_size:.2f} MB")

        return True

    except Exception as e:
        print(f"ONNX export failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# TENSORFLOW LITE EXPORT

In [ ]:
def export_to_tflite_from_pt(model_dir, tflite_path):
    try:
        import tensorflow as tf
        from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

        print("Converting XtremeDistil NSFW/Safe PyTorch model to TensorFlow...")

        # Load and convert to TensorFlow
        tokenizer = AutoTokenizer.from_pretrained(model_dir)
        tf_model = TFAutoModelForSequenceClassification.from_pretrained(
            model_dir,
            from_pt=True
        )

        max_len = 256

        @tf.function(input_signature=[
            tf.TensorSpec([None, max_len], tf.int32, name="input_ids"),
            tf.TensorSpec([None, max_len], tf.int32, name="attention_mask")
        ])
        def serving_fn(input_ids, attention_mask):
            outputs = tf_model(input_ids, attention_mask=attention_mask, training=False)
            return {"logits": outputs.logits}

        # Save as TensorFlow SavedModel
        tf_saved_model_dir = os.path.join(model_dir, "tf_saved_model")
        tf.saved_model.save(tf_model, tf_saved_model_dir, signatures={"serving_default": serving_fn})
        print(f"Saved intermediate TensorFlow model to {tf_saved_model_dir}")

        # Convert to TFLite with optimizations
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_saved_model_dir)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        # Additional optimizations for mobile deployment
        converter.target_spec.supported_types = [tf.float16]

        tflite_model = converter.convert()

        # Save TFLite model
        os.makedirs(os.path.dirname(tflite_path), exist_ok=True)
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)

        # Get model size
        model_size = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"TFLite NSFW/Safe classification model successfully exported to: {tflite_path}")
        print(f"TFLite model size: {model_size:.2f} MB")

        return True

    except Exception as e:
        print(f"TensorFlow Lite export failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# MAIN

In [ ]:
def main():
    print("Starting XtremeDistil NSFW/Safe Content Classification Pipeline")
    print("="*70)

    # Configuration - Updated paths for NSFW/Safe classification
    MODEL_NAME = "microsoft/xtremedistil-l6-h384-uncased"
    OUTPUT_DIR = "models/xtremedistil_nsfw_safe_classification"
    ONNX_PATH = "models/xtremedistil_nsfw_safe_model.onnx"
    TFLITE_PATH = "models/xtremedistil_nsfw_safe_model.tflite"

    # Create output directories
    os.makedirs("models", exist_ok=True)
    os.makedirs("metrics", exist_ok=True)

    print(f"Using model: {MODEL_NAME}")
    print(f"Output directory: {OUTPUT_DIR}")
    print("Task: Binary classification (Safe vs NSFW content)")

    # Step 1: Load dataset
    df = load_dataset()
    num_labels = len(df['label'].unique())
    print(f"Number of classes detected: {num_labels}")

    # Step 2: Split dataset
    X_train, X_val, y_train, y_val = split_dataset(df)

    # Step 3: Create tokenized datasets
    train_dataset, val_dataset, tokenizer = create_tokenized_datasets(
        X_train, X_val, y_train, y_val, MODEL_NAME
    )

    # Step 4: Train model
    trainer, model = train_model(
        train_dataset, val_dataset, tokenizer, MODEL_NAME, OUTPUT_DIR, num_labels
    )

    # Step 5: Evaluate model
    accuracy, report = evaluate_model(trainer, X_val, y_val, OUTPUT_DIR, num_labels)

    # Step 6: Export to ONNX
    onnx_success = export_to_onnx(OUTPUT_DIR, ONNX_PATH)

    # Step 7: Export to TFLite
    tflite_success = export_to_tflite_from_pt(OUTPUT_DIR, TFLITE_PATH)

    # Final output
    print("\n" + "="*70)
    print("XtremeDistil NSFW/Safe Content Classification Training Complete!")

    if onnx_success:
        print(f"✅ ONNX model: {ONNX_PATH}")
    else:
        print("❌ ONNX export: FAILED")

    if tflite_success:
        print(f"✅ TFLite model: {TFLITE_PATH}")
    else:
        print("❌ TFLite export: FAILED")

    print(f"\nModel checkpoints: {OUTPUT_DIR}")
    print(f"Metrics: metrics/xtremedistil_nsfw_safe_metrics.txt")
    print(f"Final validation accuracy: {accuracy:.4f}")
    print("\nModel can classify content as:")
    print("- 0: Safe content")
    print("- 1: NSFW content")

# INFERENCE

In [ ]:
def test_inference(model_dir, test_texts=None):
    if test_texts is None:
        test_texts = [
            # Safe content examples
            "I absolutely love this product, it's amazing!",
            "This is a family-friendly educational video about science.",
            "Great customer service and fast delivery.",
            "The movie is appropriate for all age groups.",
            "This workspace follows safety guidelines.",

            # Spanish safe content
            "Me encanta este producto, es fantástico.",
            "Contenido educativo apropiado para toda la familia.",

            # Tagalog safe content
            "Napakaganda ng produktong ito, sulit na sulit!",
            "Magandang palabas para sa buong pamilya.",

            # French safe content
            "J'adore ce produit, il est formidable!",
            "Contenu familial et éducatif.",

            # NSFW content examples
            "This contains explicit adult content not suitable for minors.",
            "Inappropriate sexual references and mature themes.",
            "Contains graphic violence and disturbing imagery.",
            "Adult-oriented material with explicit language.",
            "Mature content with sexual themes.",
            "Violent content not suitable for children.",
        ]

    print("\nTesting trained XtremeDistil NSFW/Safe classification model...")

    try:
        # Load model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        tokenizer = AutoTokenizer.from_pretrained(model_dir)

        model.eval()

        # Determine label names based on number of labels
        num_labels = model.config.num_labels
        if num_labels == 2:
            label_names = ["Safe", "NSFW"]  # Updated for NSFW/Safe classification
        elif num_labels == 3:
            label_names = ["Positive", "Neutral", "Negative"]
        else:
            label_names = [f"Class_{i}" for i in range(num_labels)]

        for i, text in enumerate(test_texts):
            # Tokenize
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=128,
                padding=True
            )

            # Predict
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predicted_class = torch.argmax(predictions, dim=-1).item()
                confidence = predictions[0][predicted_class].item()

            # Map predictions to labels
            label = label_names[predicted_class] if predicted_class < len(label_names) else f"Class_{predicted_class}"

            print(f"Text {i+1}: '{text}'")
            print(f"  -> {label} (confidence: {confidence:.4f})")

            # Show all probabilities
            probs_str = ", ".join([f"{label_names[j] if j < len(label_names) else f'Class_{j}'}={predictions[0][j].item():.3f}"
                                 for j in range(num_labels)])
            print(f"  Probabilities: {probs_str}")
            print()

    except Exception as e:
        print(f"Inference test failed: {e}")
        import traceback
        traceback.print_exc()

# PROGRAM EXECUTION

In [ ]:
if __name__ == "__main__":
    try:
        # Run the main pipeline
        main()

        # Optional: Test inference
        test_inference("models/xtremedistil_nsfw_safe_classification")

    except KeyboardInterrupt:
        print("\nTraining interrupted by user.")
    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()

    print("\nProgram execution completed.")

Starting XtremeDistil NSFW/Safe Content Classification Pipeline
Using model: microsoft/xtremedistil-l6-h384-uncased
Output directory: models/xtremedistil_nsfw_safe_classification
Task: Binary classification (Safe vs NSFW content)
Loading dataset from dataset.csv...
Dataset loaded successfully. Shape: (45, 2)
Dataset validation complete. Clean shape: (45, 2)
Number of classes: 2
Binary classification detected - assuming NSFW/Safe task
Label distribution:
  0 (Safe): 25 samples
  1 (NSFW): 20 samples
Number of classes detected: 2
Splitting dataset: 20.0% for validation...
Training set: 36 samples
Validation set: 9 samples
Loading XtremeDistil tokenizer and creating tokenized datasets...


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Tokenization complete!
Sample tokenized text length: 11
Initializing XtremeDistil model for multilingual classification training...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-100062013.py:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Metrics saved to metrics/xtremedistil_nsfw_safe_metrics.txt
Validation Accuracy: 1.0000
F1-Score (Macro): 1.0000
Exporting XtremeDistil NSFW/Safe classification model to ONNX format...


/tmp/ipython-input-314980411.py:20: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
/usr/local/lib/python3.12/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause th

ONNX model exported to: models/xtremedistil_nsfw_safe_model.onnx
ONNX model size: 86.80 MB
Converting XtremeDistil NSFW/Safe PyTorch model to TensorFlow...


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Saved intermediate TensorFlow model to models/xtremedistil_nsfw_safe_classification/tf_saved_model
TFLite NSFW/Safe classification model successfully exported to: models/xtremedistil_nsfw_safe_model.tflite
TFLite model size: 43.28 MB

XtremeDistil NSFW/Safe Content Classification Training Complete!
✅ ONNX model: models/xtremedistil_nsfw_safe_model.onnx
✅ TFLite model: models/xtremedistil_nsfw_safe_model.tflite

Model checkpoints: models/xtremedistil_nsfw_safe_classification
Metrics: metrics/xtremedistil_nsfw_safe_metrics.txt
Final validation accuracy: 1.0000

Model can classify content as:
- 0: Safe content
- 1: NSFW content

Testing trained XtremeDistil NSFW/Safe classification model...
Text 1: 'I absolutely love this product, it's amazing!'
  -> Safe (confidence: 0.5414)
  Probabilities: Safe=0.541, NSFW=0.459

Text 2: 'This is a family-friendly educational video about science.'
  -> Safe (confidence: 0.5467)
  Probabilities: Safe=0.547, NSFW=0.453

Text 3: 'Great customer service an

In [ ]:
# Create zip of entire content folder
!zip -r /content/colab_content_microsoft.zip /content/


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.09.19/ (stored 0%)
  adding: content/.config/logs/2025.09.19/13.39.59.525129.log (deflated 92%)
  adding: content/.config/logs/2025.09.19/13.40.42.120946.log (deflated 56%)
  adding: content/.config/logs/2025.09.19/13.40.32.438351.log (deflated 58%)
  adding: content/.config/logs/2025.09.19/13.40.41.341666.log (deflated 57%)
  adding: content/.config/logs/2025.09.19/13.40.30.940050.log (deflated 87%)
  adding: content/.config/logs/2025.09.19/13.40.21.959342.log (deflated 58%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (store